# 🔒 Section 17: Security Policies API Testing

**Comprehensive testing of VeevaVault Security Policies capabilities**

Security policies in Vault allow you to create and manage password policies for users, controlling password requirements, expiration periods, reuse policies, security question policies, and delegated authentication via Salesforce.com™. Security policies apply across all Vaults in a multi-Vault domain.

## 🎯 Testing Scope

### Core Security Policies Endpoints
- **GET** `/api/{version}/metadata/objects/securitypolicies` - Retrieve Security Policy Metadata
- **GET** `/api/{version}/objects/securitypolicies` - Retrieve All Security Policies  
- **GET** `/api/{version}/objects/securitypolicies/{security_policy_name}` - Retrieve Security Policy

### Key Features Tested
- ✅ **Metadata Discovery** - Get security policy object structure and field definitions
- ✅ **Policy Listing** - Retrieve all available security policies in the vault
- ✅ **Policy Details** - Get detailed configuration for specific security policies
- ✅ **Authentication Types** - Check supported authentication methods
- ✅ **Password Requirements** - Validate password policy settings
- ✅ **Security Settings** - Review expiration, history, and security question policies

### Implementation Notes
Based on Section 15 and 16 successful patterns:
- Uses `VaultClient` with proper authentication flow
- Implements read-only operations for safety
- Provides intelligent fallbacks for missing data
- Uses direct API calls instead of VQL where appropriate

---

In [1]:
# 📦 Security Policies Testing Framework Setup
import sys
import os
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')

from veevavault.client.vault_client import VaultClient
from veevavault.services.queries.query_service import QueryService
import time
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional, Any
import json

# Enhanced test result tracking
@dataclass
class APITestResult:
    """Enhanced test result with detailed tracking"""
    name: str
    success: bool
    duration: float
    error_message: Optional[str] = None
    response_data: Optional[Dict] = None
    endpoint: Optional[str] = None
    method: str = "GET"
    status_code: Optional[int] = None

class SecurityPoliciesAPITester:
    """Security Policies API Testing Framework for VeevaVault"""
    
    def __init__(self, client: VaultClient):
        self.client = client
        self.query_service = None  # Will be initialized after authentication
        self.results: List[APITestResult] = []
        self.start_time = time.time()
        
        # Security Policies specific tracking
        self.security_policies_found = []
        self.metadata_structure = None
        self.policy_details_cache = {}
        self.authentication_types_found = []
        self.total_api_calls_tested = 0
        
    def init_services(self):
        """Initialize services after authentication"""
        self.query_service = QueryService(self.client)
        
    def log_test(self, name: str, success: bool, duration: float, 
                 error_msg: str = None, response_data: Dict = None, 
                 endpoint: str = None, method: str = "GET", status_code: int = None):
        """Log test result with enhanced tracking"""
        result = APITestResult(
            name=name, success=success, duration=duration,
            error_message=error_msg, response_data=response_data,
            endpoint=endpoint, method=method, status_code=status_code
        )
        self.results.append(result)
        
        # Real-time feedback
        status = "✅ PASS" if success else "❌ FAIL"
        print(f"{status} [{len(self.results):02d}]: {name} ({duration:.2f}s)")
        if error_msg:
            print(f"        ❌ Error: {error_msg}")
        elif response_data and success:
            # Show key metrics for successful calls
            if 'responseStatus' in response_data:
                print(f"        📊 Status: {response_data['responseStatus']}")

    def test_security_policy_metadata(self):
        """Test GET /api/{version}/metadata/objects/securitypolicies"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the metadata URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/metadata/objects/securitypolicies"
            
            # Make direct API call using the client's session
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                self.metadata_structure = data
                
                # Extract useful metadata information
                metadata_info = {}
                if 'metadata' in data:
                    metadata = data['metadata']
                    metadata_info['object_name'] = metadata.get('name', 'Unknown')
                    metadata_info['description'] = metadata.get('description', 'No description')
                    metadata_info['properties_count'] = len(metadata.get('properties', []))
                    metadata_info['objects_count'] = len(metadata.get('objects', []))
                    
                    # Check for policy details and security settings
                    for prop in metadata.get('properties', []):
                        if prop.get('name') == 'policy_details__v':
                            metadata_info['has_policy_details'] = True
                        elif prop.get('name') == 'policy_security_settings__v':
                            metadata_info['has_security_settings'] = True
                
                self.log_test(
                    "Security Policy Metadata", True, response_time,
                    response_data=data,
                    endpoint="/metadata/objects/securitypolicies", method="GET"
                )
                
                print(f"        📋 Object: {metadata_info.get('object_name', 'Unknown')}")
                print(f"        📝 Description: {metadata_info.get('description', 'No description')}")
                print(f"        🔢 Properties: {metadata_info.get('properties_count', 0)}")
                print(f"        📦 Sub-objects: {metadata_info.get('objects_count', 0)}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "Security Policy Metadata", False, response_time,
                    error_msg=error_msg,
                    endpoint="/metadata/objects/securitypolicies", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "Security Policy Metadata", False, time.time() - start_time,
                error_msg=str(e), endpoint="/metadata/objects/securitypolicies", method="GET"
            )
            return None

    def test_all_security_policies(self):
        """Test GET /api/{version}/objects/securitypolicies"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the policies list URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/securitypolicies"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Extract security policies information
                policies = data.get('security_policies__v', [])
                self.security_policies_found = policies
                
                self.log_test(
                    "All Security Policies", True, response_time,
                    response_data=data,
                    endpoint="/objects/securitypolicies", method="GET"
                )
                
                print(f"        📋 Policies found: {len(policies)}")
                for policy in policies[:5]:  # Show first 5
                    name = policy.get('name__v', 'Unknown')
                    label = policy.get('label__v', 'No label')
                    print(f"        🔒 {name}: {label}")
                    
                if len(policies) > 5:
                    print(f"        ... and {len(policies) - 5} more policies")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "All Security Policies", False, response_time,
                    error_msg=error_msg,
                    endpoint="/objects/securitypolicies", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "All Security Policies", False, time.time() - start_time,
                error_msg=str(e), endpoint="/objects/securitypolicies", method="GET"
            )
            return None

    def test_specific_security_policy(self, policy_name: str):
        """Test GET /api/{version}/objects/securitypolicies/{security_policy_name}"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the specific policy URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/securitypolicies/{policy_name}"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Cache the policy details
                self.policy_details_cache[policy_name] = data
                
                # Extract policy information
                policy_info = {}
                if 'security_policy__v' in data:
                    policy = data['security_policy__v']
                    
                    # Policy details
                    if 'policy_details__v' in policy:
                        details = policy['policy_details__v']
                        policy_info['name'] = details.get('name__v', 'Unknown')
                        policy_info['label'] = details.get('label__v', 'No label')
                        policy_info['is_active'] = details.get('is_active__v', False)
                    
                    # Security settings
                    if 'policy_security_settings__v' in policy:
                        settings = policy['policy_security_settings__v']
                        
                        # Authentication type
                        if 'authentication_type__v' in settings:
                            auth_type = settings['authentication_type__v']
                            auth_type_name = auth_type.get('name__v', 'Unknown')
                            if auth_type_name not in self.authentication_types_found:
                                self.authentication_types_found.append(auth_type_name)
                            policy_info['auth_type'] = auth_type_name
                        
                        # Password requirements
                        policy_info['require_number'] = settings.get('passwords_require_number__v', False)
                        policy_info['require_uppercase'] = settings.get('passwords_require_uppercase_letter__v', False)
                        policy_info['require_nonalpha'] = settings.get('passwords_require_nonalpha_char__v', False)
                        policy_info['min_length'] = settings.get('min_password_length__v', 'Not set')
                        policy_info['expiration'] = settings.get('password_expiration__v', 'Not set')
                        policy_info['history_reuse'] = settings.get('password_history_reuse__v', 'Not set')
                
                self.log_test(
                    f"Security Policy Details ({policy_name})", True, response_time,
                    response_data=data,
                    endpoint=f"/objects/securitypolicies/{policy_name}", method="GET"
                )
                
                print(f"        🔒 Policy: {policy_info.get('label', 'Unknown')} ({policy_info.get('name', 'Unknown')})")
                print(f"        ✅ Active: {policy_info.get('is_active', False)}")
                print(f"        🔐 Auth Type: {policy_info.get('auth_type', 'Unknown')}")
                print(f"        📏 Min Length: {policy_info.get('min_length', 'Not set')}")
                print(f"        🔢 Require Number: {policy_info.get('require_number', False)}")
                print(f"        🔠 Require Uppercase: {policy_info.get('require_uppercase', False)}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    f"Security Policy Details ({policy_name})", False, response_time,
                    error_msg=error_msg,
                    endpoint=f"/objects/securitypolicies/{policy_name}", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                f"Security Policy Details ({policy_name})", False, time.time() - start_time,
                error_msg=str(e), endpoint=f"/objects/securitypolicies/{policy_name}", method="GET"
            )
            return None

    def test_all_available_policies(self):
        """Test all available security policies found in the vault"""
        if not self.security_policies_found:
            print("        ⚠️ No security policies found to test")
            return
            
        print(f"        🔍 Testing {len(self.security_policies_found)} security policies...")
        
        for policy in self.security_policies_found:
            policy_name = policy.get('name__v')
            if policy_name:
                print(f"        📋 Testing policy: {policy_name}")
                self.test_specific_security_policy(policy_name)
    
    def generate_summary(self):
        """Generate comprehensive test summary"""
        total_time = time.time() - self.start_time
        success_count = sum(1 for r in self.results if r.success)
        total_tests = len(self.results)
        
        print("\n" + "="*60)
        print("🔒 SECURITY POLICIES API TESTING - FINAL SUMMARY")
        print("="*60)
        
        print(f"\n📊 TEST SUMMARY")
        print("="*40)
        print(f"✅ SUCCESS: {success_count}")
        print(f"❌ FAILED: {total_tests - success_count}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.client.sessionId else '❌ Inactive'}")
        
        print(f"\n📊 DETAILED RESULTS")
        print("-" * 40)
        for i, result in enumerate(self.results, 1):
            status = "✅ PASS" if result.success else "❌ FAIL"
            print(f"{status} [{i:02d}]: {result.name} ({result.duration:.2f}s)")
            if result.success and result.response_data:
                if 'vault_id' in result.response_data:
                    print(f"        🏢 Vault ID: {result.response_data['vault_id']}")
            elif not result.success:
                print(f"        ❌ Error: {result.error_message}")
        
        print(f"\n🏆 FINAL STATISTICS")
        print("-" * 40)
        print(f"📈 Success Rate: {success_count}/{total_tests} ({(success_count/total_tests*100):.1f}%)")
        print(f"⏱️ Total Execution Time: {total_time:.2f}s")
        print(f"🔒 Security Policies Found: {len(self.security_policies_found)}")
        print(f"🔐 Authentication Types: {len(self.authentication_types_found)}")
        print(f"📋 Policy Details Cached: {len(self.policy_details_cache)}")
        print(f"📞 Total API Calls: {self.total_api_calls_tested}")
        
        if self.client.vaultDNS and self.client.sessionId:
            print(f"\n🏢 VAULT STATE")
            print("-" * 40)
            print(f"🔗 Connected to: {self.client.vaultDNS}")
            if hasattr(self.client, 'vaultId'):
                print(f"🆔 Vault ID: {self.client.vaultId}")
            if hasattr(self.client, 'vaultUserName'):
                print(f"👤 User: {self.client.vaultUserName}")
            print(f"🔐 Session: Active")
        
        print(f"\n🎯 API COVERAGE")
        print("-" * 40)
        print("✅ GET /metadata/objects/securitypolicies")
        print("✅ GET /objects/securitypolicies")
        print("✅ GET /objects/securitypolicies/{policy_name}")
        
        print(f"\n🔍 DISCOVERED SECURITY FEATURES")
        print("-" * 40)
        if self.security_policies_found:
            print(f"🔒 Security Policies: {len(self.security_policies_found)}")
            for policy in self.security_policies_found[:3]:
                print(f"   • {policy.get('label__v', 'Unknown')} ({policy.get('name__v', 'Unknown')})")
            if len(self.security_policies_found) > 3:
                print(f"   ... and {len(self.security_policies_found) - 3} more")
        
        if self.authentication_types_found:
            print(f"🔐 Authentication Types: {', '.join(self.authentication_types_found)}")
        
        print(f"\n✅ Security Policies API testing completed successfully!")
        print(f"🔒 All operations were read-only and safe")
        print(f"📊 Comprehensive coverage of all security policy endpoints")
        
        # Export summary for continuation
        summary_data = {
            'section': 17,
            'name': 'Security Policies',
            'success_rate': f"{(success_count/total_tests*100):.1f}%",
            'total_tests': total_tests,
            'successful_tests': success_count,
            'total_time': f"{total_time:.2f}s",
            'policies_found': len(self.security_policies_found),
            'api_calls_tested': self.total_api_calls_tested
        }
        
        print(f"\n📦 Section summary exported for continuation: {summary_data['success_rate']} success rate")
        print("="*60)
        
        return summary_data

print("✅ Security Policies API Testing Framework loaded!")
print("🔒 Ready for comprehensive security policies testing")
print("🎯 Features: Metadata discovery, policy listing, detailed configuration retrieval")

✅ Security Policies API Testing Framework loaded!
🔒 Ready for comprehensive security policies testing
🎯 Features: Metadata discovery, policy listing, detailed configuration retrieval


In [2]:
# 🔐 Load credentials 
exec(open('/Users/mp/Documents/Code/VeevaTools/veevatools/veevavault/test_credentials.py').read())

# Extract credentials from TEST_VAULTS
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_DNS = vault_config["URL"]
USERNAME = vault_config["username"]
PASSWORD = vault_config["password"]

print("🔐 Credentials loaded successfully!")
print(f"🏢 Vault DNS: {VAULT_DNS}")
print(f"👤 Username: {USERNAME}")
print("✅ Ready for Security Policies API testing!")

🔐 Credentials loaded successfully!
🏢 Vault DNS: https://vv-consulting-michael-mastermind.veevavault.com/
👤 Username: veevatools@vv-consulting.com
✅ Ready for Security Policies API testing!


In [3]:
# 🚀 Initialize Security Policies API Testing Framework
from veevavault.client.vault_client import VaultClient

# Create client instance (reuse the working client if available)
if 'client' in locals() and client.sessionId:
    print("🔄 Reusing authenticated client from previous section...")
    security_client = client  # Reuse the working authenticated client
else:
    print("🆕 Creating new client instance...")
    security_client = VaultClient()
    security_client.vaultDNS = VAULT_DNS

# Initialize Security Policies tester
security_tester = SecurityPoliciesAPITester(security_client)

print("🚀 Security Policies API Testing Framework initialized!")
print("🔒 Ready to test VeevaVault Security Policies capabilities")
print("🎯 Framework features:")
print("   ✅ Security policy metadata discovery")
print("   ✅ All security policies listing")
print("   ✅ Individual policy details retrieval")
print("   ✅ Authentication type analysis")
print("   ✅ Password policy examination")

🆕 Creating new client instance...
🚀 Security Policies API Testing Framework initialized!
🔒 Ready to test VeevaVault Security Policies capabilities
🎯 Framework features:
   ✅ Security policy metadata discovery
   ✅ All security policies listing
   ✅ Individual policy details retrieval
   ✅ Authentication type analysis
   ✅ Password policy examination


In [4]:
# 🔑 Authenticate with VeevaVault (if needed)
if not security_client.sessionId:
    print("🔑 Authenticating with VeevaVault...")

    start_time = time.time()
    try:
        # Authenticate using the correct parameters
        auth_response = security_client.authenticate(
            vaultURL=VAULT_DNS,
            vaultUserName=USERNAME,
            vaultPassword=PASSWORD,
            if_return=True
        )
        auth_duration = time.time() - start_time
        
        # VaultClient returns sessionId on success instead of responseStatus
        if auth_response and auth_response.get('sessionId'):
            vault_id = auth_response.get('vaultId')
            session_id = auth_response.get('sessionId')
            
            print(f"✅ Authentication successful!")
            print(f"🏢 Vault ID: {vault_id}")
            print(f"🔐 Session established: {session_id[:20]}...")
            
            # Initialize query services now that we're authenticated
            security_tester.init_services()
            
            # Log successful authentication
            security_tester.log_test(
                "Authentication", True, auth_duration,
                response_data={'vault_id': vault_id, 'session_id': session_id},
                endpoint="/auth", method="POST"
            )
            
        else:
            error_msg = "Authentication failed - no session ID returned"
            print(f"❌ {error_msg}")
            if auth_response:
                print(f"Response: {auth_response}")
            security_tester.log_test(
                "Authentication", False, auth_duration,
                error_msg=error_msg, endpoint="/auth", method="POST"
            )
            
    except Exception as e:
        auth_duration = time.time() - start_time
        error_msg = f"Authentication exception: {str(e)}"
        print(f"❌ {error_msg}")
        security_tester.log_test(
            "Authentication", False, auth_duration,
            error_msg=error_msg, endpoint="/auth", method="POST"
        )
else:
    print("✅ Using existing authenticated session from previous section!")
    print(f"🏢 Vault ID: {security_client.vaultId}")
    print(f"🔐 Session: {security_client.sessionId[:20]}...")
    
    # Initialize services with existing session
    security_tester.init_services()
    
    # Log reused authentication
    security_tester.log_test(
        "Authentication (Reused)", True, 0.0,
        response_data={'vault_id': security_client.vaultId, 'session_reused': True},
        endpoint="/auth", method="REUSE"
    )

print(f"\n🎯 Ready to test Security Policies APIs!")

🔑 Authenticating with VeevaVault...
✅ Authentication successful!
🏢 Vault ID: 92425
🔐 Session established: 976553B26EB9811B2FF1...
✅ PASS [01]: Authentication (0.62s)

🎯 Ready to test Security Policies APIs!


In [5]:
# 🔍 Test 1: Security Policy Metadata Discovery
print("🔍 Testing Security Policy Metadata Discovery...")
print("📋 GET /api/v25.2/metadata/objects/securitypolicies")

metadata_result = security_tester.test_security_policy_metadata()

print(f"\n✅ Security policy metadata discovery completed!")
if metadata_result:
    print(f"📋 Metadata structure retrieved and cached")
else:
    print(f"⚠️ Metadata discovery failed - check vault permissions")

🔍 Testing Security Policy Metadata Discovery...
📋 GET /api/v25.2/metadata/objects/securitypolicies
✅ PASS [02]: Security Policy Metadata (0.18s)
        📊 Status: SUCCESS
        📋 Object: security_policy
        📝 Description: Security Policy
        🔢 Properties: 2
        📦 Sub-objects: 2

✅ Security policy metadata discovery completed!
📋 Metadata structure retrieved and cached


In [6]:
# 📋 Test 2: All Security Policies Listing
print("🔍 Testing All Security Policies Listing...")
print("📋 GET /api/v25.2/objects/securitypolicies")

policies_result = security_tester.test_all_security_policies()

print(f"\n✅ All security policies listing completed!")
if policies_result:
    print(f"🔒 Found {len(security_tester.security_policies_found)} security policies")
    if security_tester.security_policies_found:
        print(f"📋 Ready to test individual policy details")
else:
    print(f"⚠️ Security policies listing failed - check vault permissions")

🔍 Testing All Security Policies Listing...
📋 GET /api/v25.2/objects/securitypolicies
✅ PASS [03]: All Security Policies (0.18s)
        📊 Status: SUCCESS
        📋 Policies found: 11
        🔒 71: Basic
        🔒 69: Classic
        🔒 13435: Cross-Domain Security Policy
        🔒 92488: Locked
        🔒 4425: No PW Expiration
        ... and 6 more policies

✅ All security policies listing completed!
🔒 Found 11 security policies
📋 Ready to test individual policy details


In [7]:
# 🔒 Test 3: Individual Security Policy Details
print("🔍 Testing Individual Security Policy Details...")
print("📋 GET /api/v25.2/objects/securitypolicies/{policy_name}")

# Test all available policies
security_tester.test_all_available_policies()

print(f"\n✅ Individual security policy details testing completed!")
print(f"🔒 Tested {len(security_tester.policy_details_cache)} policy details")
print(f"🔐 Authentication types found: {security_tester.authentication_types_found}")
print(f"📞 Total API calls made: {security_tester.total_api_calls_tested}")

🔍 Testing Individual Security Policy Details...
📋 GET /api/v25.2/objects/securitypolicies/{policy_name}
        🔍 Testing 11 security policies...
        📋 Testing policy: 71
✅ PASS [04]: Security Policy Details (71) (0.18s)
        📊 Status: SUCCESS
        🔒 Policy: Basic (71)
        ✅ Active: True
        🔐 Auth Type: Password
        📏 Min Length: 8
        🔢 Require Number: True
        🔠 Require Uppercase: True
        📋 Testing policy: 69
✅ PASS [05]: Security Policy Details (69) (0.19s)
        📊 Status: SUCCESS
        🔒 Policy: Classic (69)
        ✅ Active: True
        🔐 Auth Type: Password
        📏 Min Length: 8
        🔢 Require Number: True
        🔠 Require Uppercase: False
        📋 Testing policy: 13435
✅ PASS [06]: Security Policy Details (13435) (0.17s)
        📊 Status: SUCCESS
        🔒 Policy: Cross-Domain Security Policy (13435)
        ✅ Active: True
        🔐 Auth Type: CrossDomainAuth
        📏 Min Length: 8
        🔢 Require Number: False
        🔠 Require

In [8]:
# 📊 Generate Comprehensive Test Summary
print("📊 Generating comprehensive Security Policies test summary...")

security_summary = security_tester.generate_summary()

# Store results for next section
section_17_results = {
    'security_tester': security_tester,
    'summary': security_summary,
    'client': security_client,
    'policies_found': security_tester.security_policies_found,
    'metadata_structure': security_tester.metadata_structure
}

print(f"\n🎯 Section 17 (Security Policies) completed successfully!")
print(f"🔒 All security policy endpoints tested!")
print(f"📋 Ready to continue to Section 18...")

📊 Generating comprehensive Security Policies test summary...

🔒 SECURITY POLICIES API TESTING - FINAL SUMMARY

📊 TEST SUMMARY
✅ SUCCESS: 14
❌ FAILED: 0
⏱️ Total time: 21.07s
🔐 Session: ✅ Active

📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.62s)
        🏢 Vault ID: 92425
✅ PASS [02]: Security Policy Metadata (0.18s)
✅ PASS [03]: All Security Policies (0.18s)
✅ PASS [04]: Security Policy Details (71) (0.18s)
✅ PASS [05]: Security Policy Details (69) (0.19s)
✅ PASS [06]: Security Policy Details (13435) (0.17s)
✅ PASS [07]: Security Policy Details (92488) (0.16s)
✅ PASS [08]: Security Policy Details (4425) (0.19s)
✅ PASS [09]: Security Policy Details (30663) (0.18s)
✅ PASS [10]: Security Policy Details (2263) (0.17s)
✅ PASS [11]: Security Policy Details (2257) (0.18s)
✅ PASS [12]: Security Policy Details (43905) (0.17s)
✅ PASS [13]: Security Policy Details (29989) (0.19s)
✅ PASS [14]: Security Policy Details (65307) (0.17s)

🏆 FINAL STATISTICS


# 🔒 Section 17: Security Policies API Testing

**Comprehensive testing of VeevaVault Security Policies capabilities**

Security policies in Vault allow you to create and manage password policies for users, controlling password requirements, expiration periods, reuse policies, security question policies, and delegated authentication via Salesforce.com™. Security policies apply across all Vaults in a multi-Vault domain.

## 🎯 Testing Scope

### Core Security Policies Endpoints
- **GET** `/api/{version}/metadata/objects/securitypolicies` - Retrieve Security Policy Metadata
- **GET** `/api/{version}/objects/securitypolicies` - Retrieve All Security Policies  
- **GET** `/api/{version}/objects/securitypolicies/{security_policy_name}` - Retrieve Security Policy

### Key Features Tested
- ✅ **Metadata Discovery** - Get security policy object structure and field definitions
- ✅ **Policy Listing** - Retrieve all available security policies in the vault
- ✅ **Policy Details** - Get detailed configuration for specific security policies
- ✅ **Authentication Types** - Check supported authentication methods
- ✅ **Password Requirements** - Validate password policy settings
- ✅ **Security Settings** - Review expiration, history, and security question policies

### Implementation Notes
Based on Section 15 and 16 successful patterns:
- Uses `VaultClient` with proper authentication flow
- Implements read-only operations for safety
- Provides intelligent fallbacks for missing data
- Uses direct API calls instead of VQL where appropriate

---

In [1]:
# 📦 Security Policies Testing Framework Setup
import sys
import os
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')

from veevavault.client.vault_client import VaultClient
from veevavault.services.queries.query_service import QueryService
import time
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional, Any
import json

# Enhanced test result tracking
@dataclass
class APITestResult:
    """Enhanced test result with detailed tracking"""
    name: str
    success: bool
    duration: float
    error_message: Optional[str] = None
    response_data: Optional[Dict] = None
    endpoint: Optional[str] = None
    method: str = "GET"
    status_code: Optional[int] = None

class SecurityPoliciesAPITester:
    """Security Policies API Testing Framework for VeevaVault"""
    
    def __init__(self, client: VaultClient):
        self.client = client
        self.query_service = None  # Will be initialized after authentication
        self.results: List[APITestResult] = []
        self.start_time = time.time()
        
        # Security Policies specific tracking
        self.security_policies_found = []
        self.metadata_structure = None
        self.policy_details_cache = {}
        self.authentication_types_found = []
        self.total_api_calls_tested = 0
        
    def init_services(self):
        """Initialize services after authentication"""
        self.query_service = QueryService(self.client)
        
    def log_test(self, name: str, success: bool, duration: float, 
                 error_msg: str = None, response_data: Dict = None, 
                 endpoint: str = None, method: str = "GET", status_code: int = None):
        """Log test result with enhanced tracking"""
        result = APITestResult(
            name=name, success=success, duration=duration,
            error_message=error_msg, response_data=response_data,
            endpoint=endpoint, method=method, status_code=status_code
        )
        self.results.append(result)
        
        # Real-time feedback
        status = "✅ PASS" if success else "❌ FAIL"
        print(f"{status} [{len(self.results):02d}]: {name} ({duration:.2f}s)")
        if error_msg:
            print(f"        ❌ Error: {error_msg}")
        elif response_data and success:
            # Show key metrics for successful calls
            if 'responseStatus' in response_data:
                print(f"        📊 Status: {response_data['responseStatus']}")
        
print("✅ Security Policies API Testing Framework loaded!")
print("🔒 Ready for comprehensive security policies testing")
print("🎯 Features: Metadata discovery, policy listing, detailed configuration retrieval")
<VSCode.Cell id="#VSC-78590085" language="markdown">
## 🔐 Configuration and Credentials

Load vault credentials and configure the testing environment.

**Required:** Ensure `test_credentials.py` exists with valid vault access credentials.

SyntaxError: invalid syntax (2826415108.py, line 71)

In [ ]:
# 🔐 Load credentials 
exec(open('/Users/mp/Documents/Code/VeevaTools/veevatools/veevavault/test_credentials.py').read())

# Extract credentials from TEST_VAULTS
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_DNS = vault_config["URL"]
USERNAME = vault_config["username"]
PASSWORD = vault_config["password"]

print("🔐 Credentials loaded successfully!")
print(f"🏢 Vault DNS: {VAULT_DNS}")
print(f"👤 Username: {USERNAME}")
print("✅ Ready for Security Policies API testing!")

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🔒 Security Policies Testing Framework

Comprehensive Security Policies API testing framework with all three core endpoints.

In [ ]:
    def test_security_policy_metadata(self):
        """Test GET /api/{version}/metadata/objects/securitypolicies"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the metadata URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/metadata/objects/securitypolicies"
            
            # Make direct API call using the client's session
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                self.metadata_structure = data
                
                # Extract useful metadata information
                metadata_info = {}
                if 'metadata' in data:
                    metadata = data['metadata']
                    metadata_info['object_name'] = metadata.get('name', 'Unknown')
                    metadata_info['description'] = metadata.get('description', 'No description')
                    metadata_info['properties_count'] = len(metadata.get('properties', []))
                    metadata_info['objects_count'] = len(metadata.get('objects', []))
                    
                    # Check for policy details and security settings
                    for prop in metadata.get('properties', []):
                        if prop.get('name') == 'policy_details__v':
                            metadata_info['has_policy_details'] = True
                        elif prop.get('name') == 'policy_security_settings__v':
                            metadata_info['has_security_settings'] = True
                
                self.log_test(
                    "Security Policy Metadata", True, response_time,
                    response_data=data,
                    endpoint="/metadata/objects/securitypolicies", method="GET"
                )
                
                print(f"        📋 Object: {metadata_info.get('object_name', 'Unknown')}")
                print(f"        📝 Description: {metadata_info.get('description', 'No description')}")
                print(f"        🔢 Properties: {metadata_info.get('properties_count', 0)}")
                print(f"        📦 Sub-objects: {metadata_info.get('objects_count', 0)}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "Security Policy Metadata", False, response_time,
                    error_msg=error_msg,
                    endpoint="/metadata/objects/securitypolicies", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "Security Policy Metadata", False, time.time() - start_time,
                error_msg=str(e), endpoint="/metadata/objects/securitypolicies", method="GET"
            )
            return None

    def test_all_security_policies(self):
        """Test GET /api/{version}/objects/securitypolicies"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the policies list URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/securitypolicies"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Extract security policies information
                policies = data.get('security_policies__v', [])
                self.security_policies_found = policies
                
                self.log_test(
                    "All Security Policies", True, response_time,
                    response_data=data,
                    endpoint="/objects/securitypolicies", method="GET"
                )
                
                print(f"        📋 Policies found: {len(policies)}")
                for policy in policies[:5]:  # Show first 5
                    name = policy.get('name__v', 'Unknown')
                    label = policy.get('label__v', 'No label')
                    print(f"        🔒 {name}: {label}")
                    
                if len(policies) > 5:
                    print(f"        ... and {len(policies) - 5} more policies")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "All Security Policies", False, response_time,
                    error_msg=error_msg,
                    endpoint="/objects/securitypolicies", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "All Security Policies", False, time.time() - start_time,
                error_msg=str(e), endpoint="/objects/securitypolicies", method="GET"
            )
            return None

    def test_specific_security_policy(self, policy_name: str):
        """Test GET /api/{version}/objects/securitypolicies/{security_policy_name}"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the specific policy URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/securitypolicies/{policy_name}"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Cache the policy details
                self.policy_details_cache[policy_name] = data
                
                # Extract policy information
                policy_info = {}
                if 'security_policy__v' in data:
                    policy = data['security_policy__v']
                    
                    # Policy details
                    if 'policy_details__v' in policy:
                        details = policy['policy_details__v']
                        policy_info['name'] = details.get('name__v', 'Unknown')
                        policy_info['label'] = details.get('label__v', 'No label')
                        policy_info['is_active'] = details.get('is_active__v', False)
                    
                    # Security settings
                    if 'policy_security_settings__v' in policy:
                        settings = policy['policy_security_settings__v']
                        
                        # Authentication type
                        if 'authentication_type__v' in settings:
                            auth_type = settings['authentication_type__v']
                            auth_type_name = auth_type.get('name__v', 'Unknown')
                            if auth_type_name not in self.authentication_types_found:
                                self.authentication_types_found.append(auth_type_name)
                            policy_info['auth_type'] = auth_type_name
                        
                        # Password requirements
                        policy_info['require_number'] = settings.get('passwords_require_number__v', False)
                        policy_info['require_uppercase'] = settings.get('passwords_require_uppercase_letter__v', False)
                        policy_info['require_nonalpha'] = settings.get('passwords_require_nonalpha_char__v', False)
                        policy_info['min_length'] = settings.get('min_password_length__v', 'Not set')
                        policy_info['expiration'] = settings.get('password_expiration__v', 'Not set')
                        policy_info['history_reuse'] = settings.get('password_history_reuse__v', 'Not set')
                
                self.log_test(
                    f"Security Policy Details ({policy_name})", True, response_time,
                    response_data=data,
                    endpoint=f"/objects/securitypolicies/{policy_name}", method="GET"
                )
                
                print(f"        🔒 Policy: {policy_info.get('label', 'Unknown')} ({policy_info.get('name', 'Unknown')})")
                print(f"        ✅ Active: {policy_info.get('is_active', False)}")
                print(f"        🔐 Auth Type: {policy_info.get('auth_type', 'Unknown')}")
                print(f"        📏 Min Length: {policy_info.get('min_length', 'Not set')}")
                print(f"        🔢 Require Number: {policy_info.get('require_number', False)}")
                print(f"        🔠 Require Uppercase: {policy_info.get('require_uppercase', False)}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    f"Security Policy Details ({policy_name})", False, response_time,
                    error_msg=error_msg,
                    endpoint=f"/objects/securitypolicies/{policy_name}", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                f"Security Policy Details ({policy_name})", False, time.time() - start_time,
                error_msg=str(e), endpoint=f"/objects/securitypolicies/{policy_name}", method="GET"
            )
            return None

    def test_all_available_policies(self):
        """Test all available security policies found in the vault"""
        if not self.security_policies_found:
            print("        ⚠️ No security policies found to test")
            return
            
        print(f"        🔍 Testing {len(self.security_policies_found)} security policies...")
        
        for policy in self.security_policies_found:
            policy_name = policy.get('name__v')
            if policy_name:
                print(f"        📋 Testing policy: {policy_name}")
                self.test_specific_security_policy(policy_name)
    
    def generate_summary(self):
        """Generate comprehensive test summary"""
        total_time = time.time() - self.start_time
        success_count = sum(1 for r in self.results if r.success)
        total_tests = len(self.results)
        
        print("\n" + "="*60)
        print("🔒 SECURITY POLICIES API TESTING - FINAL SUMMARY")
        print("="*60)
        
        print(f"\n📊 TEST SUMMARY")
        print("="*40)
        print(f"✅ SUCCESS: {success_count}")
        print(f"❌ FAILED: {total_tests - success_count}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.client.sessionId else '❌ Inactive'}")
        
        print(f"\n📊 DETAILED RESULTS")
        print("-" * 40)
        for i, result in enumerate(self.results, 1):
            status = "✅ PASS" if result.success else "❌ FAIL"
            print(f"{status} [{i:02d}]: {result.name} ({result.duration:.2f}s)")
            if result.success and result.response_data:
                if 'vault_id' in result.response_data:
                    print(f"        🏢 Vault ID: {result.response_data['vault_id']}")
            elif not result.success:
                print(f"        ❌ Error: {result.error_message}")
        
        print(f"\n🏆 FINAL STATISTICS")
        print("-" * 40)
        print(f"📈 Success Rate: {success_count}/{total_tests} ({(success_count/total_tests*100):.1f}%)")
        print(f"⏱️ Total Execution Time: {total_time:.2f}s")
        print(f"🔒 Security Policies Found: {len(self.security_policies_found)}")
        print(f"🔐 Authentication Types: {len(self.authentication_types_found)}")
        print(f"📋 Policy Details Cached: {len(self.policy_details_cache)}")
        print(f"📞 Total API Calls: {self.total_api_calls_tested}")
        
        if self.client.vaultDNS and self.client.sessionId:
            print(f"\n🏢 VAULT STATE")
            print("-" * 40)
            print(f"🔗 Connected to: {self.client.vaultDNS}")
            if hasattr(self.client, 'vaultId'):
                print(f"🆔 Vault ID: {self.client.vaultId}")
            if hasattr(self.client, 'vaultUserName'):
                print(f"👤 User: {self.client.vaultUserName}")
            print(f"🔐 Session: Active")
        
        print(f"\n🎯 API COVERAGE")
        print("-" * 40)
        print("✅ GET /metadata/objects/securitypolicies")
        print("✅ GET /objects/securitypolicies")
        print("✅ GET /objects/securitypolicies/{policy_name}")
        
        print(f"\n🔍 DISCOVERED SECURITY FEATURES")
        print("-" * 40)
        if self.security_policies_found:
            print(f"🔒 Security Policies: {len(self.security_policies_found)}")
            for policy in self.security_policies_found[:3]:
                print(f"   • {policy.get('label__v', 'Unknown')} ({policy.get('name__v', 'Unknown')})")
            if len(self.security_policies_found) > 3:
                print(f"   ... and {len(self.security_policies_found) - 3} more")
        
        if self.authentication_types_found:
            print(f"🔐 Authentication Types: {', '.join(self.authentication_types_found)}")
        
        print(f"\n✅ Security Policies API testing completed successfully!")
        print(f"🔒 All operations were read-only and safe")
        print(f"📊 Comprehensive coverage of all security policy endpoints")
        
        # Export summary for continuation
        summary_data = {
            'section': 17,
            'name': 'Security Policies',
            'success_rate': f"{(success_count/total_tests*100):.1f}%",
            'total_tests': total_tests,
            'successful_tests': success_count,
            'total_time': f"{total_time:.2f}s",
            'policies_found': len(self.security_policies_found),
            'api_calls_tested': self.total_api_calls_tested
        }
        
        print(f"\n📦 Section summary exported for continuation: {summary_data['success_rate']} success rate")
        print("="*60)
        
        return summary_data

print("✅ Security Policies API Testing Framework loaded!")
print("🔒 Ready for comprehensive security policies testing")
print("🎯 Features: Metadata discovery, policy listing, detailed configuration retrieval")

✅ Baseline API testing framework initialized


## 🚀 Client Initialization and Authentication

Initialize the VaultClient and perform authentication using the successful Section 15/16 patterns.

In [ ]:
# 🚀 Initialize Security Policies API Testing Framework
from veevavault.client.vault_client import VaultClient

# Create client instance (reuse the working client if available)
if 'client' in locals() and client.sessionId:
    print("🔄 Reusing authenticated client from previous section...")
    security_client = client  # Reuse the working authenticated client
else:
    print("🆕 Creating new client instance...")
    security_client = VaultClient()
    security_client.vaultDNS = VAULT_DNS

# Initialize Security Policies tester
security_tester = SecurityPoliciesAPITester(security_client)

print("🚀 Security Policies API Testing Framework initialized!")
print("🔒 Ready to test VeevaVault Security Policies capabilities")
print("🎯 Framework features:")
print("   ✅ Security policy metadata discovery")
print("   ✅ All security policies listing")
print("   ✅ Individual policy details retrieval")
print("   ✅ Authentication type analysis")
print("   ✅ Password policy examination")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com/
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing baseline test framework...
✅ Baseline framework ready


In [ ]:
# 🔑 Authenticate with VeevaVault (if needed)
if not security_client.sessionId:
    print("🔑 Authenticating with VeevaVault...")

    start_time = time.time()
    try:
        # Authenticate using the correct parameters
        auth_response = security_client.authenticate(
            vaultURL=VAULT_DNS,
            vaultUserName=USERNAME,
            vaultPassword=PASSWORD,
            if_return=True
        )
        auth_duration = time.time() - start_time
        
        # VaultClient returns sessionId on success instead of responseStatus
        if auth_response and auth_response.get('sessionId'):
            vault_id = auth_response.get('vaultId')
            session_id = auth_response.get('sessionId')
            
            print(f"✅ Authentication successful!")
            print(f"🏢 Vault ID: {vault_id}")
            print(f"🔐 Session established: {session_id[:20]}...")
            
            # Initialize query services now that we're authenticated
            security_tester.init_services()
            
            # Log successful authentication
            security_tester.log_test(
                "Authentication", True, auth_duration,
                response_data={'vault_id': vault_id, 'session_id': session_id},
                endpoint="/auth", method="POST"
            )
            
        else:
            error_msg = "Authentication failed - no session ID returned"
            print(f"❌ {error_msg}")
            if auth_response:
                print(f"Response: {auth_response}")
            security_tester.log_test(
                "Authentication", False, auth_duration,
                error_msg=error_msg, endpoint="/auth", method="POST"
            )
            
    except Exception as e:
        auth_duration = time.time() - start_time
        error_msg = f"Authentication exception: {str(e)}"
        print(f"❌ {error_msg}")
        security_tester.log_test(
            "Authentication", False, auth_duration,
            error_msg=error_msg, endpoint="/auth", method="POST"
        )
else:
    print("✅ Using existing authenticated session from previous section!")
    print(f"🏢 Vault ID: {security_client.vaultId}")
    print(f"🔐 Session: {security_client.sessionId[:20]}...")
    
    # Initialize services with existing session
    security_tester.init_services()
    
    # Log reused authentication
    security_tester.log_test(
        "Authentication (Reused)", True, 0.0,
        response_data={'vault_id': security_client.vaultId, 'session_reused': True},
        endpoint="/auth", method="REUSE"
    )

print(f"\n🎯 Ready to test Security Policies APIs!")

🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active


## 🔒 Security Policies API Testing

Execute comprehensive tests for all Security Policies endpoints.

### Test Sequence:
1. **Metadata Discovery** - GET `/metadata/objects/securitypolicies`
2. **All Policies Listing** - GET `/objects/securitypolicies`
3. **Individual Policy Details** - GET `/objects/securitypolicies/{policy_name}`

All tests are read-only and safe for production environments.

In [ ]:
# 🔍 Test 1: Security Policy Metadata Discovery
print("🔍 Testing Security Policy Metadata Discovery...")
print("📋 GET /api/v25.2/metadata/objects/securitypolicies")

metadata_result = security_tester.test_security_policy_metadata()

print(f"\n✅ Security policy metadata discovery completed!")
if metadata_result:
    print(f"📋 Metadata structure retrieved and cached")
else:
    print(f"⚠️ Metadata discovery failed - check vault permissions")

✅ Session utilities ready: get_session_info(), end_session()
🔍 SESSION INFO
Active: ✅ Yes
Session ID: EFFDF861DBECA76326C1...
Vault ID: 92425
Tests Run: 1


In [ ]:
# 📋 Test 2: All Security Policies Listing
print("🔍 Testing All Security Policies Listing...")
print("📋 GET /api/v25.2/objects/securitypolicies")

policies_result = security_tester.test_all_security_policies()

print(f"\n✅ All security policies listing completed!")
if policies_result:
    print(f"🔒 Found {len(security_tester.security_policies_found)} security policies")
    if security_tester.security_policies_found:
        print(f"📋 Ready to test individual policy details")
else:
    print(f"⚠️ Security policies listing failed - check vault permissions")

In [ ]:
# 🔒 Test 3: Individual Security Policy Details
print("🔍 Testing Individual Security Policy Details...")
print("📋 GET /api/v25.2/objects/securitypolicies/{policy_name}")

# Test all available policies
security_tester.test_all_available_policies()

print(f"\n✅ Individual security policy details testing completed!")
print(f"🔒 Tested {len(security_tester.policy_details_cache)} policy details")
print(f"🔐 Authentication types found: {security_tester.authentication_types_found}")
print(f"📞 Total API calls made: {security_tester.total_api_calls_tested}")

In [ ]:
# 📊 Generate Comprehensive Test Summary
print("📊 Generating comprehensive Security Policies test summary...")

security_summary = security_tester.generate_summary()

# Store results for next section
section_17_results = {
    'security_tester': security_tester,
    'summary': security_summary,
    'client': security_client,
    'policies_found': security_tester.security_policies_found,
    'metadata_structure': security_tester.metadata_structure
}

print(f"\n🎯 Section 17 (Security Policies) completed successfully!")
print(f"🔒 All security policy endpoints tested!")
print(f"📋 Ready to continue to Section 18...")